In [54]:
import pandas as pd
pd.set_option('display.max_columns', None) 
import numpy as np
import pickle

import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [55]:
input_path = 'H:\RediMinds\VCQI'
df_filter2_codes = pd.read_csv(input_path+"\VCQI_clean.csv")

In [56]:
# sepearte target variable from predictors for furthur processing
X = df_filter2_codes.drop(columns=['INTRA-OP COMPLICATION'])
Y = df_filter2_codes['INTRA-OP COMPLICATION']
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2852 entries, 0 to 2851
Data columns (total 34 columns):
PROC NAME                          2852 non-null int64
GENDER                             2852 non-null int64
AGE AT SURGERY                     2852 non-null float64
MARITAL STATUS                     2852 non-null int64
RACE                               2852 non-null int64
WEIGHT                             2852 non-null float64
HEIGHT                             2852 non-null float64
BMI                                2852 non-null float64
CLINICAL SIZE (mm)                 2852 non-null float64
SYMPTOMS                           2852 non-null int64
SOLITARY KIDNEY                    2852 non-null int64
BILATERALITY OF TUMOR              2852 non-null int64
SIDE OF TUMOR                      2852 non-null int64
SIDE OF SURGERY                    2852 non-null int64
FACE                               2852 non-null int64
TUMOR lOCATION                     2852 non-null int64
PRE

In [57]:
# Create dummy variables
with open (input_path+'\cat_col', 'rb') as fp:
    cat_col = pickle.load(fp)


nn_data = X.copy()
for col in cat_col:
    if not col == 'INTRA-OP COMPLICATION':
        nn_data = nn_data.merge(pd.get_dummies(X[col], prefix=col), how = 'left', left_index = True, right_index =True) # create dummy variables
        nn_data = nn_data.drop(labels = col, axis = 'columns') # drop original variables

In [58]:
nn_data.head()

,AGE AT SURGERY,WEIGHT,HEIGHT,BMI,CLINICAL SIZE (mm),PRE-OP HB,PRE-OP HT,PRE-OP WBC,PRE-OP CREAT,PRE-OP EGFR,NO OF LESIONS,PROC NAME_0,GENDER_0,GENDER_1,GENDER_2,MARITAL STATUS_0,MARITAL STATUS_1,MARITAL STATUS_2,MARITAL STATUS_3,MARITAL STATUS_4,RACE_0,RACE_1,RACE_2,RACE_3,RACE_4,SYMPTOMS_0,SYMPTOMS_1,SYMPTOMS_2,SYMPTOMS_3,SOLITARY KIDNEY_0,SOLITARY KIDNEY_1,SOLITARY KIDNEY_2,BILATERALITY OF TUMOR_0,BILATERALITY OF TUMOR_1,BILATERALITY OF TUMOR_2,SIDE OF TUMOR_0,SIDE OF TUMOR_1,SIDE OF TUMOR_2,SIDE OF TUMOR_3,SIDE OF SURGERY_0,SIDE OF SURGERY_1,SIDE OF SURGERY_2,SIDE OF SURGERY_3,FACE_0,FACE_1,FACE_2,TUMOR lOCATION_0,TUMOR lOCATION_1,TUMOR lOCATION_2,TUMOR lOCATION_3,POLAR LOCATION_0,POLAR LOCATION_1,POLAR LOCATION_2,RENAL SINUS_0,RENAL SINUS_1,RENAL SINUS_2,RENAL SINUS_3,RENAL SINUS_4,EXOPHYTIC RATE_0,EXOPHYTIC RATE_1,EXOPHYTIC RATE_2,EXOPHYTIC RATE_3,CLINICAL SIZE GROUP_0,CLINICAL SIZE GROUP_1,CLINICAL SIZE GROUP_2,CLINICAL SIZE GROUP_3,CT_0,CT_1,CT_2,CT_3,CT_4,CT_5,CT_6,CN_0,CN_1,CN_2,CN_3,CM_0,CM_1,CM_2,CM_3,RADIUS (maximal diameter in cm)_0,RADIUS (maximal diameter in cm)_1,RADIUS (maximal diameter in cm)_2,RADIUS (maximal diameter in cm)_3,EXOPHYTIC/ENDOPHYTIC PROPERTIES_0,EXOPHYTIC/ENDOPHYTIC PROPERTIES_1,EXOPHYTIC/ENDOPHYTIC PROPERTIES_2,EXOPHYTIC/ENDOPHYTIC PROPERTIES_3,ANTERIOR OR POSTERIOR_0,ANTERIOR OR POSTERIOR_1,ANTERIOR OR POSTERIOR_2,ANTERIOR OR POSTERIOR_3,ASA SCORE_0,ASA SCORE_1,ASA SCORE_2,ASA SCORE_3,ASA SCORE_4,PARTIAL NEPHRO INDICATION_0,PARTIAL NEPHRO INDICATION_1,PARTIAL NEPHRO INDICATION_2,PARTIAL NEPHRO INDICATION_3
0,50.0,57.00,168.0,20.20,7.0,13.2,50.0,7800.0,0.63,100.030000,1.0,1,1,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0
1,69.0,68.60,161.3,26.37,74.0,14.7,44.2,6400.0,0.80,71.119672,1.0,1,1,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,1,0,0,1,0,1,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1
2,83.0,83.65,164.0,31.10,80.0,12.5,38.9,5590.0,0.97,54.844682,1.0,1,1,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0
3,66.0,107.00,180.0,32.78,73.0,17.0,50.8,8000.0,0.93,81.290783,1.0,1,0,1,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0
4,61.0,82.90,171.0,28.35,8.0,12.1,37.1,7710.0,1.10,68.053260,1.0,1,0,1,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,1,0,0,1,0,1,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0


In [59]:
# Split data in test and train
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(nn_data, Y, test_size=0.30, random_state=42, stratify = Y)

In [60]:
print("Number of records in trainset {}".format(len(x_train)))
print("Number records in testset {}".format(len(x_test)))

Number of records in trainset 1996
Number records in testset 856


In [61]:
sum(y_train)/len(y_train)

0.06462925851703406

In [62]:
sum(y_test)/len(y_test)

0.06425233644859812

In [63]:
# Standardize the numeric columns
from sklearn.preprocessing import StandardScaler
stdc = StandardScaler()
x_train.reset_index(drop=True, inplace=True)
y_train.reset_index(drop=True, inplace=True)
x_test.reset_index(drop=True, inplace=True)
y_test.reset_index(drop=True, inplace=True)
float_col = x_train.select_dtypes('float').columns
x_train[float_col] = pd.DataFrame(stdc.fit_transform(x_train[float_col]),columns=float_col).copy()
x_test[float_col] = pd.DataFrame(stdc.transform(x_test[float_col]),columns=float_col).copy()

H:\Anaconda3\lib\site-packages\pandas\core\frame.py:3391: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]
H:\Anaconda3\lib\site-packages\pandas\core\frame.py:3391: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


In [64]:
x_train.head()

,AGE AT SURGERY,WEIGHT,HEIGHT,BMI,CLINICAL SIZE (mm),PRE-OP HB,PRE-OP HT,PRE-OP WBC,PRE-OP CREAT,PRE-OP EGFR,NO OF LESIONS,PROC NAME_0,GENDER_0,GENDER_1,GENDER_2,MARITAL STATUS_0,MARITAL STATUS_1,MARITAL STATUS_2,MARITAL STATUS_3,MARITAL STATUS_4,RACE_0,RACE_1,RACE_2,RACE_3,RACE_4,SYMPTOMS_0,SYMPTOMS_1,SYMPTOMS_2,SYMPTOMS_3,SOLITARY KIDNEY_0,SOLITARY KIDNEY_1,SOLITARY KIDNEY_2,BILATERALITY OF TUMOR_0,BILATERALITY OF TUMOR_1,BILATERALITY OF TUMOR_2,SIDE OF TUMOR_0,SIDE OF TUMOR_1,SIDE OF TUMOR_2,SIDE OF TUMOR_3,SIDE OF SURGERY_0,SIDE OF SURGERY_1,SIDE OF SURGERY_2,SIDE OF SURGERY_3,FACE_0,FACE_1,FACE_2,TUMOR lOCATION_0,TUMOR lOCATION_1,TUMOR lOCATION_2,TUMOR lOCATION_3,POLAR LOCATION_0,POLAR LOCATION_1,POLAR LOCATION_2,RENAL SINUS_0,RENAL SINUS_1,RENAL SINUS_2,RENAL SINUS_3,RENAL SINUS_4,EXOPHYTIC RATE_0,EXOPHYTIC RATE_1,EXOPHYTIC RATE_2,EXOPHYTIC RATE_3,CLINICAL SIZE GROUP_0,CLINICAL SIZE GROUP_1,CLINICAL SIZE GROUP_2,CLINICAL SIZE GROUP_3,CT_0,CT_1,CT_2,CT_3,CT_4,CT_5,CT_6,CN_0,CN_1,CN_2,CN_3,CM_0,CM_1,CM_2,CM_3,RADIUS (maximal diameter in cm)_0,RADIUS (maximal diameter in cm)_1,RADIUS (maximal diameter in cm)_2,RADIUS (maximal diameter in cm)_3,EXOPHYTIC/ENDOPHYTIC PROPERTIES_0,EXOPHYTIC/ENDOPHYTIC PROPERTIES_1,EXOPHYTIC/ENDOPHYTIC PROPERTIES_2,EXOPHYTIC/ENDOPHYTIC PROPERTIES_3,ANTERIOR OR POSTERIOR_0,ANTERIOR OR POSTERIOR_1,ANTERIOR OR POSTERIOR_2,ANTERIOR OR POSTERIOR_3,ASA SCORE_0,ASA SCORE_1,ASA SCORE_2,ASA SCORE_3,ASA SCORE_4,PARTIAL NEPHRO INDICATION_0,PARTIAL NEPHRO INDICATION_1,PARTIAL NEPHRO INDICATION_2,PARTIAL NEPHRO INDICATION_3
0,0.373047,-0.091720,-0.034245,0.160169,-1.400980,0.010632,0.050049,0.005423,-1.085007,0.716033,-0.228715,1,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,1,0,0,1,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0
1,-0.594240,-0.091720,-0.034245,-1.636898,-0.103122,0.010632,0.050049,0.005423,-0.738895,1.469701,-0.228715,1,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,1,0,0,1,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0
2,0.131225,-0.588002,-0.071492,-0.607179,-0.103122,-0.893815,-0.751801,0.277927,0.393837,-0.558044,-0.228715,1,0,1,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,1,1,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0
3,2.065798,1.122244,2.163299,-0.244171,0.823919,-3.108788,-3.490428,-0.395680,0.173584,-0.533602,-0.228715,1,0,1,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,1,1,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0
4,-1.319705,0.189383,0.354183,-0.064464,0.082286,-1.324505,-2.158123,-0.353046,0.614090,-0.554292,-0.228715,1,0,1,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,1,0,0,1,0,1,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0


In [77]:
1/sum(y_train)

0.007751937984496124

In [79]:
1/(len(y_train)-sum(y_train))

0.0005356186395286556

In [83]:
# calculater class weight
class_weight = {0: 1/(len(y_train)-sum(y_train)), 1: 1/sum(y_train)}

In [65]:
# input layer
input_layer = keras.layers.Input(shape=(x_train.shape[1],), name = "input_layer")
x = keras.layers.Dense(256, name = 'Dense_1')(input_layer)
x = keras.layers.Dropout(0.2, name=  'Dropout_1')(x)
x = keras.layers.Dense(256, name = 'Dense_2')(x)
x = keras.layers.Dropout(0.2, name=  'Dropout_2')(x)
main_output = keras.layers.Dense(1, activation='sigmoid',name='main_output')(x)

model = keras.Model(inputs= input_layer, outputs=main_output)

# compiling the model
model.compile(optimizer=tf.optimizers.Adam(learning_rate=0.0001),
              loss='binary_crossentropy',
              metrics=[tf.keras.metrics.AUC(curve = 'ROC',name = 'AUC_ROC'),
                       tf.keras.metrics.AUC(curve = 'PR', name = 'AUC_PR')],
              )

# Keras callback. The patience parameter is the amount of epochs to check for improvement
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, mode='min',restore_best_weights=True)


model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_layer (InputLayer)     [(None, 102)]             0         
_________________________________________________________________
Dense_1 (Dense)              (None, 256)               26368     
_________________________________________________________________
Dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
Dense_2 (Dense)              (None, 256)               65792     
_________________________________________________________________
Dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
main_output (Dense)          (None, 1)                 257       
Total params: 92,417
Trainable params: 92,417
Non-trainable params: 0
_______________________________________________________

In [85]:
model.fit(x = x_train, y = y_train,
          validation_split = 0.2,
          epochs = 50,
          callbacks = [early_stop],
          class_weight=class_weight
     )


Train on 1596 samples, validate on 400 samples
Epoch 1/50
1596/1596 [==============================] - ETA: 11s - loss: 0.0012 - AUC_ROC: 0.9770 - AUC_PR: 0.769 - ETA: 0s - loss: 9.2954e-04 - AUC_ROC: 0.8847 - AUC_PR: 0.51 - ETA: 0s - loss: 7.7720e-04 - AUC_ROC: 0.8734 - AUC_PR: 0.49 - ETA: 0s - loss: 8.6749e-04 - AUC_ROC: 0.8665 - AUC_PR: 0.51 - 1s 443us/sample - loss: 8.6793e-04 - AUC_ROC: 0.8729 - AUC_PR: 0.5186 - val_loss: 0.1456 - val_AUC_ROC: 0.8670 - val_AUC_PR: 0.5504
Epoch 2/50
1596/1596 [==============================] - ETA: 0s - loss: 6.4792e-04 - AUC_ROC: 1.0000 - AUC_PR: 1.00 - ETA: 0s - loss: 9.3979e-04 - AUC_ROC: 0.8758 - AUC_PR: 0.51 - ETA: 0s - loss: 8.3788e-04 - AUC_ROC: 0.8805 - AUC_PR: 0.54 - 0s 114us/sample - loss: 8.6005e-04 - AUC_ROC: 0.8734 - AUC_PR: 0.5473 - val_loss: 0.1454 - val_AUC_ROC: 0.8645 - val_AUC_PR: 0.5492
Epoch 3/50
1596/1596 [==============================] - ETA: 0s - loss: 0.0012 - AUC_ROC: 0.8839 - AUC_PR: 0.80 - ETA: 0s - loss: 8.3319e-04 - AU

In [86]:
# predict on test set
pd.DataFrame(model.predict(x_test))

,0
0,0.004430
1,0.000536
2,0.675815
3,0.054530
4,0.240839
5,0.013469
6,0.020460
7,0.002452
8,0.020416
9,0.050097


In [87]:
results_nn = pd.DataFrame(model.predict(x_test), columns=['pred_prob'])
results_nn['pred_label'] =  results_nn['pred_prob'].apply(lambda x: 1 if x>=0.5 else 0)
results_nn['true_label'] = np.array(y_test)

In [88]:
# NeuralNetwork Score Raw Data
print("\n Model Balanced Accuracy: \n" + str(metrics.balanced_accuracy_score(results_nn['true_label'], results_nn['pred_label'])))
print("\n Confusion Matrix : \n"+str(metrics.confusion_matrix(results_nn['true_label'], results_nn['pred_label'])))
print("\n Classification Report: \n"+ str(metrics.classification_report(results_nn['true_label'], results_nn['pred_label'])))
print("\n AUC-ROC: \n"+ str(metrics.roc_auc_score(results_nn['true_label'], results_nn['pred_prob'])))


def calc_aucpr_data(result):
    y_ACTUAL = result['true_label']
    scores_prob = result['pred_prob']
    yhat = result['pred_label']
    precision, recall, thresholds = metrics.precision_recall_curve(y_ACTUAL, scores_prob, pos_label=1)
    prc_auc = metrics.auc(recall,precision)
    return prc_auc

print("\n AUC-ROC: \n"+ str(calc_aucpr_data(results_nn)))


 Model Balanced Accuracy: 
0.6617637044603337

 Confusion Matrix : 
[[798   3]
 [ 37  18]]

 Classification Report: 
              precision    recall  f1-score   support

           0       0.96      1.00      0.98       801
           1       0.86      0.33      0.47        55

    accuracy                           0.95       856
   macro avg       0.91      0.66      0.72       856
weighted avg       0.95      0.95      0.94       856


 AUC-ROC: 
0.8941550334808762

 AUC-ROC: 
0.5684775114477097


In [89]:
from sklearn.utils import resample
resample()

In [71]:
def bootstrapped_AUC(result):
    from sklearn.utils import resample
    from tqdm import tqdm

    n_iter = 10000
    roc_auc = list()
    prc_auc = list()


    for i in range(n_iter):
        result_sample = resample(result, n_samples = len(result),random_state=i)
        
        #Calculating AUROC for each sample
        y_ACTUAL= result_sample['true_label']
        scores_prob = result_sample['pred_prob']
        fpr, tpr, thresholds = metrics.roc_curve(y_ACTUAL, scores_prob, pos_label=1)
        roc_auc.append(metrics.auc(fpr, tpr))

        #calculate AUPRC for each sample
        y_ACTUAL = result_sample['true_label']
        scores_prob = result_sample['pred_prob']
        yhat = result_sample['pred_label']
        precision, recall, thresholds = metrics.precision_recall_curve(y_ACTUAL, scores_prob, pos_label=1)
        prc_auc.append(metrics.auc(recall,precision))
    
    return roc_auc, prc_auc

In [72]:
roc_auc_nn, pr_auc_nn = bootstrapped_AUC(results_nn)

In [74]:
dict = {'roc_auc_nn': roc_auc_nn,
        'pr_auc_nn': pr_auc_nn,
       }
pd.DataFrame(dict).describe(percentiles=[0.025,0.975])

,roc_auc_nn,pr_auc_nn
count,10000.000000,10000.000000
mean,0.896966,0.568454
std,0.021756,0.074140
min,0.781824,0.237723
2.5%,0.851586,0.418560
50%,0.897717,0.570469
97.5%,0.936316,0.706917
max,0.969432,0.813393


In [53]:
pd.DataFrame(roc_auc_nn).describe(percentiles=[0.025,0.975])

,0
count,10000.000000
mean,0.885172
std,0.024511
min,0.746168
2.5%,0.833608
50%,0.886348
97.5%,0.929689
max,0.965700


In [22]:
pd.DataFrame(roc_auc_nn).describe(percentiles=[0.025,0.975])

,0
count,10000.000000
mean,0.853840
std,0.028179
min,0.735458
2.5%,0.794303
50%,0.854971
97.5%,0.905273
max,0.941009
